In [1]:
# 모듈 import

import wandb
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./cascade_swinT_fpncarafe_adamW.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.img_prefix = root
# cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.img_prefix = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 8 #batch size

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/aug_set2_fpncarafe'


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=10, interval=1)


In [3]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1953: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "
/opt/conda/envs/detection/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


In [4]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 9649, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 9418  | 1 [Paper]     | 6004  | 2 [Paper pack]  | 1573  | 3 [Metal]   | 1459  | 4 [Glass]    | 1625  |
| 5 [Plastic]       | 4481  | 6 [Styrofoam] | 2187  | 7 [Plastic bag] | 7461  | 8 [Battery] | 1652  | 9 [Clothing] | 1480  |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [5]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-04-04 00:28:32,502 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth
2022-04-04 00:28:32,742 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-04-04 00:28:32,751 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-04-04 00:28:32,852 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-04-04 00:2

In [6]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-04-04 00:28:37,313 - mmdet - INFO - Start running, host: root@3c89da8245a6, work_dir: /opt/ml/level2-object-detection-level2-cv-18/Mmdetection/_MyModel/work_dirs/aug_set2_fpncarafe
2022-04-04 00:28:37,314 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) It

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


wandb: Currently logged in as: cv18 (use `wandb login --relogin` to force relogin)


RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 31.75 GiB total capacity; 23.65 GiB already allocated; 364.50 MiB free; 24.29 GiB reserved in total by PyTorch)